In [25]:
from __future__ import print_function
import numpy as np
import pandas as pd


from sklearn.ensemble  import RandomForestClassifier as rfc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression as logreg

import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.api as sma

from sklearn.feature_selection import chi2

import scipy as sp
from scipy import stats

from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

In [3]:
# the directory that contains all the files
dataDirectory = 'data/'

In [4]:
file = 'llcp2017_formatted.csv'
df = pd.read_csv(dataDirectory+file)

/home/urwa/miniconda3/envs/nlp/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (3,170,174) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.head()

,STATE FIPS CODE,FILE MONTH,INTERVIEW DATE,INTERVIEW MONTH,INTERVIEW DAY,INTERVIEW YEAR,FINAL DISPOSITION,ANNUAL SEQUENCE NUMBER,PRIMARY SAMPLING UNIT,CORRECT TELEPHONE NUMBER?,...,_IMPCAGE,_IMPCRAC,_IMPCSEX,_IMPEDUC,_IMPHOME,_IMPMRTL,_IMPNPH,_IMPSEX,_M_RACE,_URBNRRL
0,Alabama,January,1302017,01,30,2017,1100,2017000001,2017000001,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
1,Alabama,January,1122017,01,12,2017,1100,2017000002,2017000002,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
2,Alabama,January,1102017,01,10,2017,1100,2017000003,2017000003,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
3,Alabama,January,2082017,02,8,2017,1200,2017000004,2017000004,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing
4,Alabama,January,1302017,01,30,2017,1100,2017000005,2017000005,Yes,...,Not asked or Missing,Missing,Missing,Missing,Missing,Missing,NaN,Not asked or Missing,Not asked or Missing,Missing


In [6]:
print("Number of records: ",len(df))

Number of records:  450642


### Helper Functions

In [7]:
def containsKeyword(sent):
    sent = sent.lower()
    for f in filterList:
        if f in sent:
            return True
    return False

In [8]:
def normalizeNumeric(df):
    for col in df.columns:
        if np.issubdtype(df[col].dtype, np.number):
            df[col] = ( df[col] - np.mean(df[col]) ) / (np.std(df[col]))
    return df
            
#[ (c, np.sum(np.isnan(ecigdfOH[c])))  for c in ecigdfOH.columns if np.issubdtype(ecigdfOH[c].dtype, np.number) ]

In [9]:
# remove class imballance by sampling from majority class
def classImballanceDownSample(df,ycol):
    df = df.copy()
    valueCount = df[ycol].value_counts()
    print("Before Class Imballance Treatment: ")
    print(valueCount)
    classes = valueCount.index
    counts = valueCount.values
    minClassSize = np.min(counts)
    for clas in classes:
        df1 = df[df[ycol]==clas]
        df2 = df[df[ycol]!=clas]
        
        df1 = df1.sample(n=minClassSize, random_state=50)
        df = df1.append(df2)
    #shuffling the dataframe
    df = df.sample(frac=1).reset_index(drop=True)
    print("After Class Imballance Treatment: ")
    print(df[ycol].value_counts())
    return df  

In [10]:
def labelEncodeCategoricalFeatures(DF):
    labelencoder = LabelEncoder()
    df = DF.copy()
    for c in df.columns:
        if df[c].dtype.name == 'object':
            df[c] = labelencoder.fit_transform(df[c])
    return df

In [11]:
def performChiSquareDependencyTest(df,label):
    xcols = [c for c in df.columns if c != label]
    X = df[xcols]
    Y = df[label]
    res = chi2(labelEncodeCategoricalFeatures(X),Y)
    resDf = pd.DataFrame({'Variable':xcols, 'Chi':res[0], 'P_value':res[1]})
    resDf = resDf.sort_values(by=['Chi'], ascending=False).reset_index(drop=True)
    return resDf

In [12]:
def getKbestChiFeatures(df,label,k):
    xcols = [c for c in df.columns if c != label]
    X = df[xcols]
    Y = df[label]
    res = chi2(labelEncodeCategoricalFeatures(X),Y)
    kcols = [col for ch,col in sorted(zip(res[0],xcols),reverse=True)]
    resDf = df[kcols[:k]+[label]]
    return resDf

### Create Computed Column Dataframe

In [13]:
# Find the columns that contain smoking key word
filterList = ['smok','cig']    
smokecols = [c for c in df.columns if containsKeyword(c)]
smokecols

## Explore the calculated and computed columns
filterList = ['computed', 'calculated']    
calcols = [c for c in df.columns if containsKeyword(c)]
print("Number of calculated/computed columns: ",len(calcols))

# create a datframe that only has calculated columns
comDF = df[calcols]

Number of calculated/computed columns:  41


### Create SMoking columns

In [14]:
ecigdf = comDF.copy()
ecigdf.rename(columns={'CURRENT E-CIGARETTE USER CALCULATED VARI': 'esmoke', \
                     'CURRENT SMOKING CALCULATED VARIABLE':'smoke'},inplace = True)


mapper = {'Current E-cigarette user': 'Yes', 'Not currently using E-cigarettes': 'No' }
ecigdf['esmoke'] = ecigdf['esmoke'].map(mapper)

print("data size: ",len(ecigdf))
ecigdf = ecigdf[ecigdf.smoke.apply(lambda x: x in ['Yes','No']) ]
ecigdf = ecigdf[ecigdf.esmoke.apply(lambda x: x in ['Yes','No']) ]
print("clean data size: ",len(ecigdf))

print("Frequency distributions:")
print(ecigdf.esmoke.value_counts())
print(ecigdf.smoke.value_counts())

data size:  450642
clean data size:  427526
Frequency distributions:
No     413906
Yes     13620
Name: esmoke, dtype: int64
No     364794
Yes     62732
Name: smoke, dtype: int64


In [15]:
print("Just smoke :", np.sum((ecigdf.smoke == 'Yes') & (ecigdf.esmoke == 'No')))
print("Just esmoke:", np.sum((ecigdf.smoke == 'No') & (ecigdf.esmoke == 'Yes')))
print("SMoke Both :", np.sum((ecigdf.smoke == 'Yes') & (ecigdf.esmoke == 'Yes')))
print("Smoke none :", np.sum((ecigdf.smoke == 'No') & (ecigdf.esmoke == 'No')))

Just smoke : 55523
Just esmoke: 6411
SMoke Both : 7209
Smoke none : 358383


### Data Cleansing

In [16]:
# remove non smokers
ecigdf = ecigdf[ (ecigdf.smoke == 'Yes') | (ecigdf.esmoke == 'Yes') ]
print("clean data size: ",len(ecigdf))

clean data size:  69143


In [17]:
print("Records: ",len(ecigdf))
ecigdf = ecigdf.dropna()
print("Records after cleaning Nan: ",len(ecigdf))

Records:  69143
Records after cleaning Nan:  61225


In [18]:
# label is 1 for esmoker and zero for non smoker
ecigdf['label'] = ecigdf.esmoke == 'Yes'

In [19]:
excludecolumns = smokecols + ['esmoke', 'smoke']
ecigdf = ecigdf[[c for c in ecigdf.columns if c not in excludecolumns ]]
print('Columns : ',len(ecigdf.columns))
print("Frequency distributions:")
print(ecigdf.label.value_counts())

Columns :  38
Frequency distributions:
False    48998
True     12227
Name: label, dtype: int64


## Logit Analysis

In [93]:
kkk = pd.get_dummies(ecigdf.iloc[:,0:1])
[ c for c in kkk.columns if ecigdf.columns[0] in c]

['COMPUTED PHYSICAL HEALTH STATUS_1-13 days when physical health not good',
 'COMPUTED PHYSICAL HEALTH STATUS_14+ days when physical health not good',
 'COMPUTED PHYSICAL HEALTH STATUS_Don�t know/Refused/Missing',
 'COMPUTED PHYSICAL HEALTH STATUS_Zero days when physical health not good']

In [ ]:
range()

In [113]:
def filterPerfectCorrelation(df):
    cols = df.columns
    removeCols = []
    for i in range(len(cols)-1):
        for j in range(i+1,len(cols)):
            cor = stats.pearsonr(df[cols[i]].values, df[cols[j]].values)[0]
            if cor >=1 or cor <=-1:
                removeCols.append(cols[i])
    return df[[c for c in cols if c not in removeCols]]
        

In [126]:
filteredDf = getKbestChiFeatures(ecigdf,'label',50)

resCols = []
for i in range(1,len(filteredDf.columns)-1):

    X=filteredDf[[c for c in filteredDf.columns if c not in ['label'] ]]
    #print(X.corr())
    
    
    Y= filteredDf['label']
    X = pd.get_dummies(X)
    iterCols = [ c for c in X.columns if filteredDf.columns[i] in c] + resCols
    #print(filteredDf.columns[i])
    #print(len(iterCols))
    print("Columns Used: ",filteredDf.columns[i])
    X = X[iterCols]
    X = filterPerfectCorrelation(X)

    logit_mod = sma.Logit(Y, X, method='bfgs').fit()

    pred = logit_mod.predict() #> 0.5
    fpr, tpr, thresholds = metrics.roc_curve(Y, pred)
    print('AUC: ',metrics.auc(fpr, tpr))
    #print(logit_mod.summary())

    result = pd.concat([logit_mod.params, logit_mod.pvalues], axis=1).reset_index() 
    result.columns = ['var','coef','pval']
    result = result[result.pval < 0.1]
    resCols = list(result['var'].values)
    resCols = list(set(resCols))
    #print(resCols)

Columns Used:  COMPUTED FRUIT INTAKE IN TIMES PER DAY
Optimization terminated successfully.
         Current function value: 0.607426
         Iterations 6
AUC:  0.4761093703271982
Columns Used:  PNEUMONIA VACCINATION CALCULATED VARIABL
Optimization terminated successfully.
         Current function value: 0.492178
         Iterations 6
AUC:  0.5760247897513675
Columns Used:  FLU SHOT CALCULATED VARIABLE
Optimization terminated successfully.
         Current function value: 0.492146
         Iterations 6
AUC:  0.5762225009639733
Columns Used:  COMPUTED OTHER VEGETABLE INTAKE IN TIMES
Optimization terminated successfully.
         Current function value: 0.551189
         Iterations 6
AUC:  0.4341712798949106
Columns Used:  COMPUTED MENTAL HEALTH STATUS
Optimization terminated successfully.
         Current function value: 0.490990
         Iterations 6
AUC:  0.5895874975133057
Columns Used:  COMPUTED NUMBER OF DRINKS OF ALCOHOL BEV
Optimization terminated successfully.
         Current

/home/urwa/miniconda3/envs/nlp/lib/python3.6/site-packages/statsmodels/base/model.py:1092: RuntimeWarning: invalid value encountered in sqrt
  bse_ = np.sqrt(np.diag(self.cov_params()))
/home/urwa/miniconda3/envs/nlp/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/home/urwa/miniconda3/envs/nlp/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/home/urwa/miniconda3/envs/nlp/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


Columns Used:  COMPUTED DARK GREEN VEGETABLE INTAKE IN
Optimization terminated successfully.
         Current function value: 0.552323
         Iterations 6
AUC:  0.4301303028033054
Columns Used:  HIGH CHOLESTEROL CALCULATED VARIABLE
Optimization terminated successfully.
         Current function value: 0.491131
         Iterations 6
AUC:  0.5913354603601392
Columns Used:  COMPUTED PREFERRED RACE
Optimization terminated successfully.
         Current function value: 0.488522
         Iterations 7
AUC:  0.6054201757318236
Columns Used:  COMPUTED POTATO SERVINGS PER DAY
Optimization terminated successfully.
         Current function value: 0.549347
         Iterations 6
AUC:  0.44622206944898846
Columns Used:  COMPUTED RACE-ETHNICITY GROUPING
Optimization terminated successfully.
         Current function value: 0.489471
         Iterations 6
AUC:  0.5973090318600105
Columns Used:  150 MINUTE PHYSICAL ACTIVITY CALCULATED
Optimization terminated successfully.
         Current function val

/home/urwa/miniconda3/envs/nlp/lib/python3.6/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Columns Used:  HEAVY ALCOHOL CONSUMPTION  CALCULATED VA
Optimization terminated successfully.
         Current function value: 0.491876
         Iterations 6
AUC:  0.5816899612371952


In [127]:
len(resCols)

8

In [128]:
Y= ecigdf['label']
X = pd.get_dummies(ecigdf)
X = X[resCols]
X = filterPerfectCorrelation(X)
logit_mod = sma.Logit(Y, X, method='bfgs').fit()
print(logit_mod.summary())

Optimization terminated successfully.
         Current function value: 0.491876
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                61225
Model:                          Logit   Df Residuals:                    61217
Method:                           MLE   Df Model:                            7
Date:                Sat, 08 Dec 2018   Pseudo R-squ.:                 0.01624
Time:                        18:24:56   Log-Likelihood:                -30115.
converged:                       True   LL-Null:                       -30612.
                                        LLR p-value:                2.292e-210
                                                                          coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------------

In [122]:
len(pd.get_dummies(ecigdf).columns)

141

In [77]:
performChiSquareDependencyTest(filteredDf,'LIFETIME ASTHMA CALCULATED VARIABLE')

,Variable,Chi,P_value
0,COMPUTED WEIGHT IN KILOGRAMS,64371.508835,0.000000e+00
1,COMPUTED ASTHMA STATUS,20353.597711,0.000000e+00
2,COMPUTED DARK GREEN VEGETABLE INTAKE IN,2906.999637,0.000000e+00
3,COMPUTED OTHER VEGETABLE INTAKE IN TIMES,2356.947855,0.000000e+00
4,COMPUTED PHYSICAL HEALTH STATUS,1124.402987,6.902298e-245
5,COMPUTED MENTAL HEALTH STATUS,780.140037,3.933175e-170
6,COMPUTED FRUIT INTAKE IN TIMES PER DAY,551.588870,1.674938e-120
7,FLU SHOT CALCULATED VARIABLE,437.533714,9.789516e-96
8,PNEUMONIA VACCINATION CALCULATED VARIABL,377.564442,1.030204e-82
9,EVER BEEN TESTED FOR HIV CALCULATED VARI,219.470469,2.200863e-48


In [106]:
pd.get_dummies(ecigdf[['COMPUTED BODY MASS INDEX']]).corr()

,COMPUTED BODY MASS INDEX_1 or greater,COMPUTED BODY MASS INDEX_Don�t know/Refused/Missing
COMPUTED BODY MASS INDEX_1 or greater,1.0,-1.0
COMPUTED BODY MASS INDEX_Don�t know/Refused/Missing,-1.0,1.0


In [108]:
testdf = pd.get_dummies(ecigdf[['COMPUTED BODY MASS INDEX']])
stats.pearsonr(testdf.iloc[:,0].values, testdf.iloc[:,1].values)[0]

-1.0

In [115]:
filterPerfectCorrelation(pd.get_dummies(ecigdf[['COMPUTED BODY MASS INDEX']]))

,COMPUTED BODY MASS INDEX_Don�t know/Refused/Missing
5,0
20,0
26,0
27,0
29,0
34,0
48,0
56,0
59,0
62,0
